In [1]:
def lgb_as_stacker_final(pinjie_train,pinjie_test,old_train,test):
    #havent del other 5 labels,will do  when I feel it nessisary.
    import lightgbm as lgb
    from sklearn.model_selection import train_test_split
    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    submission_test = pd.DataFrame.from_dict({'id': test['id']})
    for class_name in class_names:

        train_sparse_matrix, valid_sparse_matrix, y_train__, y_valid__ = train_test_split(pinjie_train, old_train[class_name], test_size=0.05, random_state=144)
        test_sparse_matrix = pinjie_test
#         print(y_train__[class_name])
#         print(y_valid__)
        d_train = lgb.Dataset(train_sparse_matrix, label=y_train__)
#         d_train_new = lgb.Dataset(new_train_sparse_matrix, label=train_y[class_name])
        d_valid = lgb.Dataset(valid_sparse_matrix, label=y_valid__)


        watchlist = [d_train, d_valid]
        params = {'learning_rate': 0.2,
                  'application': 'binary',
                  'num_leaves': 31,
                  'verbosity': -1,
                  'metric': 'auc',
                  'data_random_seed': 2,
                  'bagging_fraction': 0.8,
                  'feature_fraction': 0.6,
                  'nthread': 4,
                  'lambda_l1': 1,
                  'lambda_l2': 1}
        rounds_lookup = {'toxic': 140,
                     'severe_toxic': 50,
                     'obscene': 80,
                     'threat': 80,
                     'insult': 70,
                     'identity_hate': 80}
        model = lgb.train(params,
                          train_set=d_train,
                          num_boost_round=rounds_lookup[class_name],
                          valid_sets=watchlist,
                          verbose_eval=10)


        submission_test[class_name] = model.predict(test_sparse_matrix)
#         submission_valid[class_name] = model.predict(valid_sparse_matrix)
    submission_test.to_csv('stack_with_lgbm_final.csv', index=False)
    
    return submission_test


In [2]:
def lgb(old_train,test,train_x,train_y,valid_x,valid_x2,valid_y):
    import gc
    import pandas as pd

    from scipy.sparse import csr_matrix, hstack

    from sklearn.feature_extraction.text import TfidfVectorizer

    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import SelectFromModel

    from sklearn.linear_model import LogisticRegression
    import lightgbm as lgb
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

#     train = pd.read_csv('train.csv').fillna(' ')
#     test = pd.read_csv('test.csv').fillna(' ')
    print('Loaded')

#     train=train[:1000]
#     test=test[:1000]
    train_text = old_train['comment_text']
    test_text = test['comment_text']

    all_text = pd.concat([train_text, test_text])

    word_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        token_pattern=r'\w{1,}',
        ngram_range=(1, 2),
        max_features=50000)
    word_vectorizer.fit(all_text)
    print('Word TFIDF 1/3')
    train_word_features = word_vectorizer.transform(train_text)
    train_word_features_new = word_vectorizer.transform(train_x)
    valid_word_features_new = word_vectorizer.transform(valid_x)


    print('Word TFIDF 2/3')
    test_word_features = word_vectorizer.transform(test_text)
    print('Word TFIDF 3/3')

    char_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='char',
        stop_words='english',
        ngram_range=(2, 6),
        max_features=50000)
    char_vectorizer.fit(all_text)
    print('Char TFIDF 1/3')
    train_char_features = char_vectorizer.transform(train_text)
    new_train_char_features = char_vectorizer.transform(train_x)
    new_valid_char_features = char_vectorizer.transform(valid_x)
    print('Char TFIDF 2/3')
    test_char_features = char_vectorizer.transform(test_text)
    print('Char TFIDF 3/3')

    train_features = hstack([train_char_features, train_word_features])
    new_train_features=hstack([new_train_char_features,train_word_features_new])
    new_valid_features=hstack([new_valid_char_features,valid_word_features_new])
    print('HStack 1/2')
    test_features = hstack([test_char_features, test_word_features])
    print('HStack 2/2')

    submission_test = pd.DataFrame.from_dict({'id': test['id']})
    submission_valid= pd.DataFrame.from_dict({'id': valid_x2['id']})
    old_train2=old_train.drop('comment_text', axis=1, inplace=False)
    del test
    del train_text
    del test_text
    del all_text
    del train_char_features
    del test_char_features
    del train_word_features
    del test_word_features
    del train_word_features_new
    del valid_word_features_new
    del new_valid_char_features
    del new_train_char_features
    
    gc.collect()
#     for i, class_name in enumerate(class_names):
#         print(i,class_name)
    for class_name in class_names:
#         print(class_name,'tset1')
        train_target = old_train2[class_name]
        model = LogisticRegression(solver='sag')
        sfm = SelectFromModel(model, threshold='1.25*mean')
#         print(train_features.shape,'tset12')
        train_sparse_matrix = sfm.fit_transform(train_features, train_target)
#         print(train_sparse_matrix.shape,'tset123')
        
        
        
#         train_sparse_matrix, valid_sparse_matrix, y_train, y_valid = train_test_split(train_sparse_matrix, train_target, test_size=0.05, random_state=144)
        test_sparse_matrix = sfm.transform(test_features)
        valid_sparse_matrix = sfm.transform(new_valid_features)
        new_train_sparse_matrix = sfm.transform(new_train_features)
#         print('tset1234')

        
#         d_train = lgb.Dataset(train_sparse_matrix, label=y_train)
        d_train_new = lgb.Dataset(new_train_sparse_matrix, label=train_y[class_name])
        d_valid = lgb.Dataset(valid_sparse_matrix, label=valid_y[class_name])
        
#         print ('12345')
        
        watchlist = [d_train_new, d_valid]
        params = {'learning_rate': 0.2,
                  'application': 'binary',
                  'num_leaves': 31,
                  'verbosity': -1,
                  'metric': 'auc',
                  'data_random_seed': 2,
                  'bagging_fraction': 0.8,
                  'feature_fraction': 0.6,
                  'nthread': 4,
                  'lambda_l1': 1,
                  'lambda_l2': 1}
        rounds_lookup = {'toxic': 140,
                     'severe_toxic': 50,
                     'obscene': 80,
                     'threat': 80,
                     'insult': 70,
                     'identity_hate': 80}
        model = lgb.train(params,
                          train_set=d_train_new,
                          num_boost_round=rounds_lookup[class_name],
                          valid_sets=watchlist,
                          verbose_eval=10)
        
#         submission[class_name] = model.predict(train_sparse_matrix)
#         print('123456')
        submission_test[class_name] = model.predict(test_sparse_matrix)
        submission_valid[class_name] = model.predict(valid_sparse_matrix)
#         print(submission_test)
    return submission_test,submission_valid


In [3]:
def lgb_get_train_test(old_train,test):
    import gc
    import pandas as pd

    from scipy.sparse import csr_matrix, hstack

    from sklearn.feature_extraction.text import TfidfVectorizer

    from sklearn.model_selection import train_test_split
    from sklearn.feature_selection import SelectFromModel

    from sklearn.linear_model import LogisticRegression
    import lightgbm as lgb
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


    class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

#     train = pd.read_csv('train.csv').fillna(' ')
#     test = pd.read_csv('test.csv').fillna(' ')
    print('Loaded')
    train_ids=old_train.id
    test_ids=test.id
    check_id=[test_ids==train_ids]
    [i for i in check_id[0] if i == False]

    
#     train=train[:1000]
#     test=test[:1000]
    train_text = old_train['comment_text']
    test_text = test['comment_text']

    all_text = pd.concat([train_text, test_text])

    word_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='word',
        token_pattern=r'\w{1,}',
        ngram_range=(1, 2),
        max_features=50000)
    word_vectorizer.fit(all_text)
    print('Word TFIDF 1/3')
    train_word_features = word_vectorizer.transform(train_text)
#     train_word_features_new = word_vectorizer.transform(train_x)
#     valid_word_features_new = word_vectorizer.transform(valid_x)


    print('Word TFIDF 2/3')
    test_word_features = word_vectorizer.transform(test_text)
    print('Word TFIDF 3/3')

    char_vectorizer = TfidfVectorizer(
        sublinear_tf=True,
        strip_accents='unicode',
        analyzer='char',
        stop_words='english',
        ngram_range=(2, 6),
        max_features=50000)
    char_vectorizer.fit(all_text)
    print('Char TFIDF 1/3')
    train_char_features = char_vectorizer.transform(train_text)
#     new_train_char_features = char_vectorizer.transform(train_x)
#     new_valid_char_features = char_vectorizer.transform(valid_x)
    print('Char TFIDF 2/3')
    test_char_features = char_vectorizer.transform(test_text)
    print('Char TFIDF 3/3')

    train_features = hstack([train_char_features, train_word_features])
#     new_train_features=hstack([new_train_char_features,train_word_features_new])
#     new_valid_features=hstack([new_valid_char_features,valid_word_features_new])
    print('HStack 1/2')
    test_features = hstack([test_char_features, test_word_features])
    print('HStack 2/2')

    submission_test = pd.DataFrame.from_dict({'id': test['id']})
#     submission_valid= pd.DataFrame.from_dict({'id': valid_x2['id']})
    old_train=old_train.drop('comment_text', axis=1, inplace=False)
    del test
    del train_text
    del test_text
    del all_text
    del train_char_features
    del test_char_features
    del train_word_features
    del test_word_features
#     del train_word_features_new

    gc.collect()
#     for i, class_name in enumerate(class_names):
#         print(i,class_name)
    for class_name in class_names:
#         print(class_name,'tset1')
        train_target = old_train[class_name]
        model = LogisticRegression(solver='sag')
        sfm = SelectFromModel(model, threshold='1.15*mean')
#         print(train_features.shape,'tset12')
        train_sparse_matrix = sfm.fit_transform(train_features, train_target)
#         print(train_sparse_matrix.shape,'tset123')
        
        
        
#         train_sparse_matrix, valid_sparse_matrix, y_train, y_valid = train_test_split(train_sparse_matrix, train_target, test_size=0.05, random_state=144)
        test_sparse_matrix = sfm.transform(test_features)


#         d_train = lgb.Dataset(train_sparse_matrix, label=train_target)


    return train_sparse_matrix,test_sparse_matrix


In [4]:
def lr_(old_train,valid_x2,test,train_x,train_y,valid_x,valid_y):
    import pandas as pd, numpy as np
    from sklearn.linear_model import LogisticRegression
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#     train = pd.read_csv('train.csv')
#     test = pd.read_csv('test.csv')
    subm = pd.read_csv('sample_submission.csv')

    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    COMMENT = 'comment_text'
    old_train[COMMENT].fillna("unknown", inplace=True)
    test[COMMENT].fillna("unknown", inplace=True)



    import re, string
    re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    def tokenize(s): return re_tok.sub(r' \1 ', s).split()

    n = old_train.shape[0]
    vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                   min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                   smooth_idf=1, sublinear_tf=1 )
    trn_term_doc = vec.fit_transform(old_train[COMMENT])
   
    train_term_doc = vec.transform(train_x)
    valid_term_doc = vec.transform(valid_x)

    test_term_doc = vec.transform(test[COMMENT])



    def pr(y_i, y):
        p = x[y==y_i].sum(0)
        return (p+1) / ((y==y_i).sum()+1)

    x = trn_term_doc
    train_x=train_term_doc
    test_x = test_term_doc
    valid_x=valid_term_doc

    submission_test = pd.DataFrame.from_dict({'id': test['id']})
    submission_valid= pd.DataFrame.from_dict({'id': valid_x2['id']})

    def get_mdl(y):
        y = y.values
        r = np.log(pr(1,y) / pr(0,y))
        m = LogisticRegression(C=4, dual=True)
        x_nb = x.multiply(r)
        return m.fit(x_nb, y), r

    preds_test = np.zeros((len(test), len(label_cols)))
    preds_valid= np.zeros((len(x_valid), len(label_cols)))

    for i, j in enumerate(label_cols):
        print('fit', j)
        m,r = get_mdl(old_train[j])
        
        preds_test[:,i] = m.predict_proba(test_x.multiply(r))[:,1]
        preds_valid[:,i] = m.predict_proba(valid_x.multiply(r))[:,1]
    df1=submission_valid
    df1_test=submission_test
    
    
    df2=pd.DataFrame(preds_valid, columns = label_cols)
    df2_test=pd.DataFrame(preds_test, columns = label_cols)
    df1.index = range(len(df1))
    df2.index = range(len(df2))
    df1_test.index = range(len(df1_test))
    df2_test.index = range(len(df2_test))
    
    submission_test=pd.concat([df1_test,df2_test],axis=1)
    submission_valid=pd.concat([df1,df2],axis=1)

    return submission_test,submission_valid


In [ ]:
def lstm_(old_train,valid_x2,test,train_x,train_y,valid_x,valid_y,top_word=2000,maxlen=50,word_vec_len=20):
    import pandas as pd

    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import Sequential
    from keras.models import Model
    from keras.layers import Dense, Embedding, Input
    from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
    from keras.preprocessing import text, sequence
#     train=pd.read_csv('train.csv')
#     test = pd.read_csv("test.csv")
    train['comment_text'] = old_train['comment_text'].str.replace('[^a-zA-Z]',' ').str.lower()
    test['comment_text'] = test['comment_text'].str.replace('[^a-zA-Z]',' ').str.lower()
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

    # train_split = train['comment_text'].str.split()
    # test_split = test['comment_text'].str.split()
#     seqence_train = train_x
#     seqence_valid = valid_x
    
    seqence_train=train["comment_text"].values
    
    
    
    seqence_test = test["comment_text"].values
    list_classes=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

    # y=train[list_classes].values


    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences

#     top_word=20000
#     maxlen=100
#     word_vec_len=50


    # 1. token and 词序号

    tokenizer=Tokenizer(top_word)
    tokenizer.fit_on_texts(list(seqence_train))
#     tokenizer.fit_on_texts(list(seqence_valid))
 
    sequence_train2=tokenizer.texts_to_sequences(train_x)
    sequence_valid2=tokenizer.texts_to_sequences(valid_x)
    # tokenizer.fit_on_texts(list(seqence_test))
    list_tokenized_test = tokenizer.texts_to_sequences(seqence_test)

#     y = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]


    # 2. pad_sequence
    pad_train=pad_sequences(sequence_train2,maxlen)
    pad_valid=pad_sequences(sequence_valid2,maxlen)
    pad_test = pad_sequences(list_tokenized_test, maxlen)

    # # 3. embedding
    # model=Sequential()
    # model.add(Embedding(mask_zero=True,input_dim=top_word,input_length=max_review_len,output_dim=word_vec_len))
    # model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
    # word2vec_train=model.predict(paded_sequence_train)
    # # word2vec_test=model.predict(X_te)

    from keras.models import Sequential
    from keras.layers import Dense, Dropout, LSTM, Bidirectional, GlobalMaxPool1D, BatchNormalization,Embedding


    import logging
    from keras.callbacks import Callback

    import sys, os, re, csv, codecs, numpy as np, pandas as pd

    from keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.layers import Dense, Input, LSTM, GRU, Embedding, Dropout, Activation, BatchNormalization
    from keras.layers import Bidirectional, GlobalMaxPool1D
    from keras.models import Model
    from keras import initializers, regularizers, constraints, optimizers, layers, callbacks


    from keras.layers import Activation,BatchNormalization
    from keras.optimizers import Adam
    embedding_layer=Embedding(input_dim=top_word,input_length=maxlen,output_dim=word_vec_len)

    bi_rnn=Sequential()
    bi_rnn.add(embedding_layer)
    bi_rnn.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1),input_shape=(maxlen,word_vec_len)))
    bi_rnn.add(GlobalMaxPool1D())
    bi_rnn.add(BatchNormalization())
    bi_rnn.add( Dense(50))
    bi_rnn.add(Activation('relu'))
    bi_rnn.add(Dropout(0.1))
    bi_rnn.add( Dense(6))
    # bi_rnn.add(BatchNormalization())
    bi_rnn.add(Activation('sigmoid'))
    # bi_rnn.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
    bi_rnn.summary()



    bi_rnn.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    # early_stopping_monitor = EarlyStopping(patience=2)
#     y = old_train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

    # lr = callbacks.LearningRateScheduler(schedule)
    filepath = "/Users/bruce/All ucl/1applied ml/kaggle/toxic/final/output/keras_nei_{val_acc:.4f}.hdf5"
    Checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False)

    # monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto'
    early = EarlyStopping(monitor='val_acc',patience=0,verbose=1,mode='auto')
    bi_rnn.fit(x=pad_train,y=train_y,validation_data=(pad_valid,valid_y),epochs=3,batch_size=64,callbacks=[early,Checkpoint])

    #model.fit(x=train_padded,y=y,validation_split=.1,epochs=3,batch_size=64)
    test_pred = bi_rnn.predict(x=pad_test)
    valid_pred = bi_rnn.predict(x=pad_valid)
    submission_test = pd.DataFrame.from_dict({'id': test['id']})
    submission_valid= pd.DataFrame.from_dict({'id': valid_x2['id']})
    
    
    df1=submission_valid
    df1_test=submission_test
    df2=pd.DataFrame(valid_pred, columns = label_cols)
    df2_test=pd.DataFrame(test_pred, columns = label_cols)
    
    
    df1.index = range(len(df1))
    df2.index = range(len(df2))
    df1_test.index = range(len(df1_test))
    df2_test.index = range(len(df2_test))

    submission_test=pd.concat([df1_test,df2_test],axis=1)
    submission_valid=pd.concat([df1,df2],axis=1)

    return submission_test,submission_valid



In [ ]:
import pandas as pd
old_train = pd.read_csv('train.csv').fillna(' ')
# old_train=old_train.head(1000)
test = pd.read_csv('test.csv').fillna(' ')
train=old_train
# test=tes

from sklearn.model_selection import KFold,train_test_split
i,P1,P2,T1,T2=0,[],[],[],[]
kf=KFold(n_splits=2)

for train_index, valid_index in kf.split(train):
#     print(valid_index)
    train_n,valid_n=train.loc[train_index],train.loc[valid_index]
    x_train=train_n['comment_text']
    x_valid=valid_n['comment_text']
    x_valid2=valid_n
    y_train=train_n[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
    y_valid=valid_n[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
    
    
    T_1,P_1=lstm_(valid_x2=x_valid2,test=test,old_train=old_train,train_x=x_train,train_y=y_train,valid_x=x_valid,valid_y=y_valid)
    print(i,'m1')

    T_2,P_2=lr_(valid_x2=x_valid2,test=test,old_train=old_train,train_x=x_train,train_y=y_train,valid_x=x_valid,valid_y=y_valid)
  
    print(i,'m2')

#     T_3,P_3=lgb(valid_x2=x_valid2,test=test,old_train=old_train,train_x=x_train,train_y=y_train,valid_x=x_valid,valid_y=y_valid)
#     print(i,'m3')

    i+=1
    P1.append(P_1)
    P2.append(P_2)
#     P.append(P_3)
    
    T1.append(T_1)
    T2.append(T_2)

Using TensorFlow backend.


In [ ]:
import numpy as np
import pandas as pd
def check_id(a,b):
    test_id=a.id
    test_id2=b.id
    check_id=[T1[0].id==T1[1].id]
    return [i for i in check_id[0] if i == False]


P1_final = pd.concat([i for i in P1], axis=0)
P2_final = pd.concat([i for i in P2], axis=0)  


check_id(P1_final,P2_final)
check_id(P1_final,old_train)
pp1=P1_final[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
pp2=P2_final[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
print(pp1.shape)
final_P=np.concatenate((pp1,pp2),axis=1)
# print(final_p.shape)
train_sparse_matrix,test_sparse_matrix=lgb_get_train_test(old_train,test)


def pinjie_train(train_sparse_matrix,final_P):
    from scipy.sparse import coo_matrix,hstack
    final_P=coo_matrix(final_P)
    final_trainnn=hstack([final_P,train_sparse_matrix])
#     final_trainnn=np.concatenate((train_sparse_matrix,final_P),axis=1)
    return final_trainnn


    
#     T.append(T_3)  

# [i for i in P1]
test_id=T1[0].id
test_id2=T1[1].id
check_id=[T1[0].id==T1[1].id]
[i for i in check_id[0] if i == False]


label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

a_=np.zeros_like(T1[0][label_cols])
print(a_.shape)
for i in T1:    
    i=i.drop('id',axis=1,inplace=False)
    a_+=i.values
    T1_final=a_/len(T1)
    
b_=np.zeros_like(T2[0][label_cols])  
print(b_.shape)
for i in T2:       
    i=i.drop('id',axis=1,inplace=False)
    b_+=i.values
    T2_final=b_/len(T2)
    
all_Ts=np.concatenate((T1_final,T2_final),axis=1)
print(all_Ts.shape)





def pinjie_test(embeded_old_test_x,all_Ts):
    from scipy.sparse import coo_matrix,hstack
    all_Ts=coo_matrix(all_Ts)
    bbb_=hstack([all_Ts,embeded_old_test_x])
#     all_P.drop('id',axis=1,inplace=True)
#     print(all_Ts.shape)
#     print(embeded_old_test_x.shape)
#     bbb_=np.concatenate((all_Ts,embeded_old_train_x),axis=1)
# #         print(all_Ts.shape)
#     print(bbb_.shape)
    return bbb_


final_train=pinjie_train(train_sparse_matrix,final_P)

final_test=pinjie_test(test_sparse_matrix,all_Ts)

stacked_test=lgb_as_stacker_final(old_train=old_train,pinjie_test=final_test,pinjie_train=final_train,test=test)




